In [3]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf

In [2]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = {}
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for idx, page in enumerate(reader.pages):
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            # Separar por instituições
            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos[f"produto_{idx_geral}"] = bloco_completo
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {len(texto_blocos)}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número[\s\S]+?([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Construção do DataFrame
data = []
for bloco_id, bloco_text in texto_blocos.items():
    row = {
        'bloco_id': bloco_id,
        'nome': get_feature(bloco_text, 'nome'),
        'nif': get_feature(bloco_text, 'nif'),
        'mes_mapa': get_feature(bloco_text, 'mes_mapa'),
        'instituicao': get_feature(bloco_text, 'instituicao'),
        'divida': get_feature(bloco_text, 'total_em_divida'),
        'litigio': get_feature(bloco_text, 'litigio'),
        'parcela': get_feature(bloco_text, 'abatido_ativo'),
        'garantias': get_feature(bloco_text, 'garantias'),
        'num_devedores': get_feature(bloco_text, 'num_devedores'),
        'prod_financeiro': get_feature(bloco_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(bloco_text, 'entrada_incumpr'),
        'dat_inicio': get_feature(bloco_text, 'dat_inicio'),
        'dat_fim': get_feature(bloco_text, 'dat_fim')
    }
    data.append(row)

df = pd.DataFrame(data)
df

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_3d5453f2-5074-47e4-b446-4caafb8ffc33.pdf

PDF: decrypted_3d5453f2-5074-47e4-b446-4caafb8ffc33.pdf | Total de páginas: 1
Total de produtos detectados em 'decrypted_3d5453f2-5074-47e4-b446-4caafb8ffc33.pdf': 0
PDF 'decrypted_3d5453f2-5074-47e4-b446-4caafb8ffc33.pdf' movido para: ./maps/decrypted/processed/decrypted_3d5453f2-5074-47e4-b446-4caafb8ffc33.pdf


""


In [4]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader
import unicodedata
from IPython.display import display

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = {}
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for idx, page in enumerate(reader.pages):
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos[f"produto_{idx_geral}"] = bloco_completo
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {len(texto_blocos)}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s\u00A0,.]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s\u00A0,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s*\n(?:.*?\n)?\d+\s+([\d\s\u00A0,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Construção do DataFrame
data = []
for bloco_id, bloco_text in texto_blocos.items():
    row = {
        'bloco_id': bloco_id,
        'nome': get_feature(bloco_text, 'nome'),
        'nif': get_feature(bloco_text, 'nif'),
        'mes_mapa': get_feature(bloco_text, 'mes_mapa'),
        'instituicao': get_feature(bloco_text, 'instituicao'),
        'divida': get_feature(bloco_text, 'total_em_divida'),
        'litigio': get_feature(bloco_text, 'litigio'),
        'parcela': get_feature(bloco_text, 'abatido_ativo'),
        'garantias': get_feature(bloco_text, 'garantias'),
        'num_devedores': get_feature(bloco_text, 'num_devedores'),
        'prod_financeiro': get_feature(bloco_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(bloco_text, 'entrada_incumpr'),
        'dat_inicio': get_feature(bloco_text, 'dat_inicio'),
        'dat_fim': get_feature(bloco_text, 'dat_fim')
    }
    data.append(row)

df = pd.DataFrame(data)

# Conversão de valores monetários
def parse_float(valor_str):
    if isinstance(valor_str, str):
        valor_str = ''.join(c for c in valor_str if not unicodedata.category(c).startswith('Z'))
        valor_str = valor_str.replace('€', '').replace(',', '.')
        try:
            return float(valor_str)
        except ValueError:
            return None
    return None

for col in ['divida', 'parcela', 'garantias']:
    df[col] = df[col].apply(parse_float)

# ... (todo o código anterior até a criação do DataFrame e parse_float permanece igual)

# Regras de perfilamento individual
def regra_sem_garantia(row):
    return pd.isna(row['garantias']) or row['garantias'] == 0.0

def regra_sem_litigio(row):
    return isinstance(row['litigio'], str) and row['litigio'].strip().lower() == 'não'

instituicoes_com_garantia = df[df['garantias'] > 0]['instituicao'].unique().tolist()

def instituicao_sem_qualquer_garantia(row):
    return row['instituicao'] not in instituicoes_com_garantia

# 1️⃣ Define perfil_individual por linha
df['perfil_individual'] = df.apply(
    lambda r: regra_sem_garantia(r) and regra_sem_litigio(r) and instituicao_sem_qualquer_garantia(r),
    axis=1
)

# 2️⃣ Soma global de dívidas válidas
soma_total_elegivel = df[df['perfil_individual']]['divida'].sum()

# 3️⃣ Resultado final
grupo_perfila = soma_total_elegivel >= 6000

# 4️⃣ Define perfila por linha
df['perfila'] = df['perfil_individual'] & grupo_perfila

# 5️⃣ Exibir DataFrame formatado
from IPython.display import display
print("\n📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':")
display(df)

# 6️⃣ Soma e status final
print(f"\n💰 Soma das dívidas elegíveis individualmente: € {soma_total_elegivel:,.2f}")
print(f"\n🧾 Resultado final: {'✅ PERFILA' if grupo_perfila else '❌ NÃO PERFILA'}")

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_ee8b0888-32a9-484f-933c-5ce90e0762dc.pdf

PDF: decrypted_ee8b0888-32a9-484f-933c-5ce90e0762dc.pdf | Total de páginas: 4
Total de produtos detectados em 'decrypted_ee8b0888-32a9-484f-933c-5ce90e0762dc.pdf': 3
PDF 'decrypted_ee8b0888-32a9-484f-933c-5ce90e0762dc.pdf' movido para: ./maps/decrypted/processed/decrypted_ee8b0888-32a9-484f-933c-5ce90e0762dc.pdf

📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':


,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim,perfil_individual,perfila
0,produto_1,ANA PAULA ALMEIDA DA SILVA ROCHA,214496007,maio de 2023,COFIDIS (0921),3667.99,Não,74.12,None,1,Crédito pessoal,2023-01-05,2022-05-11,2027-05-05,True,True
1,produto_2,ANA PAULA ALMEIDA DA SILVA ROCHA,214496007,maio de 2023,"BANCO CREDIBOM, SA (0916)",6147.06,Não,102.22,None,2,Crédito pessoal,2023-01-07,2022-05-31,2029-06-07,True,True
2,produto_3,ANA PAULA ALMEIDA DA SILVA ROCHA,214496007,maio de 2023,"UNIVERSO, IME, S.A. (7500)",707.90,Não,NaN,None,1,Cartão de crédito - com período de free-float,None,2022-07-17,9999-12-31,True,True



💰 Soma das dívidas elegíveis individualmente: € 10,522.95

🧾 Resultado final: ✅ PERFILA


In [4]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader
import unicodedata
from IPython.display import display
import json

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")
json_output_folder = "./customers"

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder, json_output_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s\u00A0,.]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s\u00A0,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s*\n(?:.*?\n)?\d+\s+([\d\s\u00A0,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = []
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for page in reader.pages:
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            nome = get_feature(full_text, 'nome')
            nif = get_feature(full_text, 'nif')
            mes_mapa = get_feature(full_text, 'mes_mapa')

            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos.append({
                        'bloco_id': f"produto_{idx_geral}",
                        'nome': nome,
                        'nif': nif,
                        'mes_mapa': mes_mapa,
                        'instituicao': nome_inst,
                        'divida': get_feature(bloco_completo, 'total_em_divida'),
                        'litigio': get_feature(bloco_completo, 'litigio'),
                        'parcela': get_feature(bloco_completo, 'abatido_ativo'),
                        'garantias': get_feature(bloco_completo, 'garantias'),
                        'num_devedores': get_feature(bloco_completo, 'num_devedores'),
                        'prod_financeiro': get_feature(bloco_completo, 'prod_financeiro'),
                        'entrada_incumpr': get_feature(bloco_completo, 'entrada_incumpr'),
                        'dat_inicio': get_feature(bloco_completo, 'dat_inicio'),
                        'dat_fim': get_feature(bloco_completo, 'dat_fim')
                    })
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {idx_geral - 1}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Criar DataFrame
df = pd.DataFrame(texto_blocos)

# Converter valores monetários
def parse_float(valor_str):
    if isinstance(valor_str, str):
        valor_str = ''.join(c for c in valor_str if not unicodedata.category(c).startswith('Z'))
        valor_str = valor_str.replace('€', '').replace(',', '.')
        try:
            return float(valor_str)
        except ValueError:
            return None
    return None

for col in ['divida', 'parcela', 'garantias']:
    df[col] = df[col].apply(parse_float)

# Regras de perfilamento
def regra_sem_garantia(row):
    return pd.isna(row['garantias']) or row['garantias'] == 0.0

def regra_sem_litigio(row):
    return isinstance(row['litigio'], str) and row['litigio'].strip().lower() == 'não'

instituicoes_com_garantia = df[df['garantias'] > 0]['instituicao'].unique().tolist()

def instituicao_sem_qualquer_garantia(row):
    return row['instituicao'] not in instituicoes_com_garantia

df['perfil_individual'] = df.apply(
    lambda r: regra_sem_garantia(r) and regra_sem_litigio(r) and instituicao_sem_qualquer_garantia(r),
    axis=1
)

soma_total_elegivel = df[df['perfil_individual']]['divida'].sum()
grupo_perfila = soma_total_elegivel >= 6000
df['perfila'] = df['perfil_individual'] & grupo_perfila

print("\n📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':")
display(df)

print(f"\n💰 Soma das dívidas elegíveis individualmente: € {soma_total_elegivel:,.2f}")
print(f"\n🧾 Resultado final: {'✅ PERFILA' if grupo_perfila else '❌ NÃO PERFILA'}")

# ... [código anterior até exportação do JSON permanece igual]

# Exportar JSON por cliente
clientes_salvos = 0
df_validos = df[df['nif'].notna()]
for nif, group in df_validos.groupby("nif"):
    json_path = os.path.join(json_output_folder, f"{nif}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(group.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    clientes_salvos += 1

print(f"\n✅ {clientes_salvos} arquivos JSON salvos em: {json_output_folder}")

# ✅ Removido: visualização do JSON de um cliente
# Se quiser exibir um JSON manualmente, você pode abrir e usar display(pd.DataFrame(...)) fora deste script.

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_4ed12942-0c72-460c-ac50-f8cdce6ff107.pdf

PDF: decrypted_4ed12942-0c72-460c-ac50-f8cdce6ff107.pdf | Total de páginas: 14
Total de produtos detectados em 'decrypted_4ed12942-0c72-460c-ac50-f8cdce6ff107.pdf': 19
PDF 'decrypted_4ed12942-0c72-460c-ac50-f8cdce6ff107.pdf' movido para: ./maps/decrypted/processed/decrypted_4ed12942-0c72-460c-ac50-f8cdce6ff107.pdf

📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':


,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim,perfil_individual,perfila
0,produto_1,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,COFIDIS (0921),4199.38,Não,0.00,NaN,2,Crédito renovável - Linha de crédito,None,1997-11-17,9999-12-31,True,True
1,produto_2,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"NOVO BANCO, S.A. (0007)",0.00,Não,NaN,NaN,2,None,None,2012-05-03,9999-12-31,False,False
2,produto_3,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"NOVO BANCO, S.A. (0007)",22838.13,Não,264.59,908.52,2,Crédito à habitação,None,2005-06-30,2032-10-05,False,False
3,produto_4,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BNP PARIBAS PERSONAL FINANCE, S.A. - SUCURSAL ...",12566.42,Não,293.71,NaN,2,Crédito pessoal,None,2020-02-19,2030-03-01,True,True
4,produto_5,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"UNIVERSO, IME, S.A. (7500)",1735.06,Não,327.96,NaN,1,Cartão de crédito - com período de free-float,2024-07-08,2018-03-29,9999-12-31,True,True
5,produto_6,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)",1718.34,Não,32.90,NaN,1,Crédito pessoal,2024-06-22,2021-11-22,2029-11-22,True,True
6,produto_7,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)",202.70,Não,3.75,NaN,1,Crédito pessoal,2024-06-22,2021-11-22,2028-11-22,True,True
7,produto_8,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)",5545.03,Não,106.45,NaN,1,Crédito pessoal,2024-06-21,2022-01-21,2030-01-21,True,True
8,produto_9,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)",646.83,Não,11.54,NaN,1,Crédito pessoal,2024-06-21,2022-01-21,2029-01-21,True,True
9,produto_10,ANABELA LIBERTO LIMÃO NOGUEIRA LOPES,189133147,dezembro de 2024,"BANCO SANTANDER TOTTA, S.A. (0018)",2711.75,Não,49.75,NaN,1,Crédito pessoal,2024-06-18,2022-03-18,2030-03-18,True,True



💰 Soma das dívidas elegíveis individualmente: € 48,220.84

🧾 Resultado final: ✅ PERFILA

✅ 1 arquivos JSON salvos em: ./customers
